In [1]:
import time
import psutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import (
    accuracy_score, f1_score, roc_auc_score,
    mean_squared_error, mean_absolute_error, r2_score,
    classification_report
)

import joblib
import os

from ucimlrepo import fetch_ucirepo


In [2]:
'''

we can choose different dataset

if using California housing dataset, please choose load_california_housing()

if using uci dataset, please choose load_data()

seed

seed we choose 777, 888, 999

'''

'\n\nwe can choose different dataset\n\nif using California housing dataset, please choose load_california_housing()\n\nif using uci dataset, please choose load_data()\n\nseed\n\nseed we choose 777, 888, 999\n\n'

In [3]:

# california_housing dataset

from sklearn.datasets import fetch_california_housing
seed = 777

def load_california_housing():
    data = fetch_california_housing()
    X = data.data 
    y = data.target 
    
    feature_names = data.feature_names
    X = pd.DataFrame(X, columns=feature_names)

    return X, y
dataset_name = "California Housing"  
task_type = "regression"

save_dir = os.path.join(r"D:\DSS5104\final\randomforest\777", dataset_name)
os.makedirs(save_dir, exist_ok=True)



In [ ]:
#load uci dataset
'''
[Covertype](https://archive.ics.uci.edu/dataset/31/covertype)

[Adult](https://archive.ics.uci.edu/dataset/2/adult)

[Bank Marketing](https://archive.ics.uci.edu/dataset/222/bank+marketing)

[Poker Hand](https://archive.ics.uci.edu/dataset/158/poker+hand)

[HIGGS](https://archive.ics.uci.edu/dataset/280/higgs)

[Wine Quality](https://archive.ics.uci.edu/dataset/186/wine+quality)
'''

seed = 999
np.random.seed(seed)

dataset_name = "Covertype" 
dataset_id = 31
task_type = "classification" # "classification" or "regression"

save_dir = os.path.join(r"D:\DSS5104\final\randomforest", dataset_name)
os.makedirs(save_dir, exist_ok=True)




In [4]:
def load_data():
    dataset = fetch_ucirepo(id=dataset_id)
    X = dataset.data.features
    y = dataset.data.targets.squeeze()

    if (X == "?").any().any():
        X = X.replace("?", np.nan)
        
        original_shape = X.shape[0]
        df = pd.concat([X, y], axis=1)
        df = df.dropna()
        removed = original_shape - df.shape[0]
        print(f"Missing value detected. Remove {removed} records")

        X = df.drop(columns=[y.name])
        y = df[y.name]

    return X, y

In [5]:

def split_features(X):
    cat_cols = X.select_dtypes(include=["object", "category"]).columns.tolist()
    num_cols = X.select_dtypes(include=["int64", "float64"]).columns.tolist()
    return num_cols, cat_cols

def build_preprocessor(num_cols, cat_cols):
    numeric_transformer = StandardScaler()
    categorical_transformer = OrdinalEncoder(
        handle_unknown="use_encoded_value",
        unknown_value=-1
    )

    preprocessor = ColumnTransformer([
        ("num", numeric_transformer, num_cols),
        ("cat", categorical_transformer, cat_cols),
    ])
    return preprocessor

In [7]:
#X, y = load_data() 
X, y = load_california_housing() #load_data()

num_cols, cat_cols = split_features(X)
preprocessor = build_preprocessor(num_cols, cat_cols)


if task_type == "classification":

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2,
        stratify=y if task_type == "classification" else None,
        random_state=seed
    )

else:
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2,
        random_state=seed
    )


print(f"dataset: train={len(X_train)}, test={len(X_test)}")


dataset: train=16512, test=4128


In [8]:
if task_type == "classification":
    model = RandomForestClassifier(n_estimators=100, max_depth=None, random_state=seed, n_jobs=-1)
else:
    model = RandomForestRegressor(n_estimators=100, max_depth=None, random_state=seed, n_jobs=-1)

pipeline = Pipeline([
    ("pre", preprocessor),
    ("rf", model),
])

In [9]:
start_time = time.time()
cpu_before = psutil.cpu_percent(interval=2)

pipeline.fit(X_train, y_train)

train_time = time.time() - start_time
cpu_after = psutil.cpu_percent(interval=2)

print(f"time: {train_time:.2f}s, CPU: {cpu_before}% → {cpu_after}%")

time: 5.30s, CPU: 18.0% → 22.0%


In [10]:
y_test_pred = pipeline.predict(X_test)

if task_type == "classification":
    
    test_accuracy = accuracy_score(y_test, y_test_pred)
    test_f1 = f1_score(y_test, y_test_pred, average="macro")  
    print("evaluation:")
    print(f"Test Accuracy: {test_accuracy:.4f}")
    print(f"Test F1 Score: {test_f1:.4f}")
    
    try:
        y_test_proba = pipeline.predict_proba(X_test)
        if len(np.unique(y_test)) == 2:
            test_auc = roc_auc_score(y_test, y_test_proba[:, 1])
        else:
            test_auc = roc_auc_score(y_test, y_test_proba, multi_class="ovr", average="macro")
        print(f"Test AUC: {test_auc:.4f}")
    except Exception as e:
        print("AUC can't caculate: ", e)
    
    print("\nClassification Report:\n", classification_report(y_test, y_test_pred))

else:
    
    test_mse = mean_squared_error(y_test, y_test_pred)
    test_rmse = test_mse ** 0.5
    test_mae = mean_absolute_error(y_test, y_test_pred)
    test_r2 = r2_score(y_test, y_test_pred)
    
    print("evaluation:")
    print(f"Test RMSE: {test_rmse:.4f}")
    print(f"Test MAE: {test_mae:.4f}")
    print(f"Test R²: {test_r2:.4f}")

evaluation:
Test RMSE: 0.4999
Test MAE: 0.3207
Test R²: 0.8061


In [12]:
# 1️⃣ 保存模型
model_filename = os.path.join(save_dir, f"{dataset_name}_random_forest_model.pkl")
joblib.dump(pipeline, model_filename)
print(f"model already saved {model_filename}")

train_metrics = {
    "train_time_seconds": train_time,
    "cpu_before_percent": cpu_before,
    "cpu_after_percent": cpu_after
}

if task_type == "classification":
    eval_results = {
        "Accuracy": test_accuracy,
        "F1_score": test_f1,
        "AUC": test_auc,
        "time": train_time,
        "cpu_before(%)": cpu_before,
        "cpu_after(%)": cpu_after,
        "cpu_occupied(%)": cpu_after - cpu_before
    }
    
elif task_type == "regression":
    eval_results = {
        "RMSE": test_rmse,
        "MAE": test_mae,
        "R2": test_r2,
        "time": train_time,
        "cpu_before(%)": cpu_before,
        "cpu_after(%)": cpu_after,
        "cpu_occupied(%)": cpu_after - cpu_before
    }

eval_df = pd.DataFrame([eval_results])



# 5️⃣ 保存到 Excel 文件
excel_filename = os.path.join(save_dir, f"{dataset_name}_model_results.xlsx")
with pd.ExcelWriter(excel_filename, engine="xlsxwriter") as writer:
    eval_df.to_excel(writer, sheet_name="Evaluation Results", index=False)

print(f"saved {excel_filename}")


model already saved D:\DSS5104\final\randomforest\777\California Housing\California Housing_random_forest_model.pkl
saved D:\DSS5104\final\randomforest\777\California Housing\California Housing_model_results.xlsx
